<p align="center">
    <img srt="ATLAS-Logo-Ref-RGB.png" title="Experiment ATLAS">
</p>

<p align="center">
 <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons">
     <img src="ATLAS-Logo-Ref-RGB.png" title="ATLAS logo" style="width:60%"></a>
</p>

<CENTER><h1>Análisis del Bosón de Higgs del experimento ATLAS en el canal de decaimiento $H \rightarrow WW^{*}\rightarrow e\nu\mu\nu$     a      $\sqrt{s}=13TeV$</h1></CENTER>

-----------
<p>Vamos a incluir unas bibliotecas estándar de C++ y ROOT</p>

In [1]:
#include <iostream>
#include <string> 
#include <vector>
#include <stdio.h> 
#include <typeinfo>
#include <sys/stat.h>
using namespace std;

---------
<p>Vamos a incluir cabeceras y funciones creadas para proposito de este análisis</p>

In [2]:
#include "Analysis/HWWAnalysisHistograms.h"
#include "Analysis/make_HWW_objects.h"
#include "Analysis/main_HWWAnalysis.C"
#include "Analysis/array_sample.C"

------------------
Se crean los folders que van a guardar los resultados del análisis y los plots finales

In [3]:
if (fileExists("Analysis/Output_HWWAnalysis") != true){
    mkdir("Analysis/Output_HWWAnalysis",S_IRWXU);
}
if (fileExists("Plotting/histograms") != true){
    mkdir("Plotting/histograms",S_IRWXU);
}

----------
<h4>Which option should I run?</h4>
<p>Optioms are</p>
<p>0=run all data and Mc one after another</p>
<p>1=run data only</p>
<p>2, 3, 4, 5, 6, 7 = run MC sample only</p>


In [4]:
int option=1;
cout<<"option: "<<option<<endl;

option: 1


---------
<h4>Should use PROOF? (will make things faster)</h4>
<p>Options are:</p>
<p>0=NO</p>
<p>1=YES</p>

In [5]:
int parallel=1;
cout<<"Option: "<<parallel<<endl;

Option: 1


-----------
<h4>Selection of samples</h4>

In [6]:
vector<string> sample;
array_samples(sample,option); //function that is in charge of assigning the names of the analyzes to be studied

-----------
<h3>Physical Analysis</h3>

In [7]:
//We define the histograms
define_histograms();

for (unsigned int i=0;i<sample.size();i++){ 
    
    fchain = main_HWWAnalysis(parallel,sample[i]);
    Init(fchain);
    Long64_t nentries = fReader.GetEntries(1);
    cout<< "Muestra a Analizar: "<< sample[i] << " # Events: " <<nentries << endl;
    
    while (fReader.Next()){
        
        // read input option
        TString foption = sample[i]; //Replacement of option=GetOption();
        
        //Scale factors
        Float_t scaleFactor = (*scaleFactor_ELE)*(*scaleFactor_MUON)*(*scaleFactor_LepTRIGGER)*(*scaleFactor_PILEUP);
        
        //MC weight
        Float_t m_mcWeight = *mcWeight;
        
        //Total Weight
        Float_t weight;
        
        if (foption.Contains("single")){
            m_mcWeight = (*mcWeight/TMath::Abs(*mcWeight)); // set to 1 or -1 single top MCs
            weight = scaleFactor*m_mcWeight;
        } else if (foption.Contains("data")){
            weight = 1;    // Make difference between data y MC
        } else {
           weight = scaleFactor*m_mcWeight; 
        }
         
        //Preselection cut for electron/muon trigger
        if (*trigE || *trigM){
            
            // Preselection of good Leptons
            int goodlep_index[2];
            int goodlep_n = 0;
            int lep_index = 0;
            
            for (unsigned int j=0; j<*lep_n; j++){
                
                TLorentzVector leptemp;
                leptemp.SetPtEtaPhiE(lep_pt[j]/1000., lep_eta[j], lep_phi[j], lep_E[j]/1000.);
                
                // Lepton is Tight----boolean indicating whether lepton satisfies tight ID reconstruction criteria
                if (lep_isTightID->at(j)){
                    
                    // standard lepton isolation requirement => strict isolation
                    if ((lep_pt[j]>15000.) && (lep_ptcone30[j]/lep_pt[j]<0.1) && (lep_etcone20[j]/lep_pt[j]<0.1)){
                       
                        // electron selection
                        if ((lep_type[j]==11) && (TMath::Abs(lep_eta[j])<2.47) && ((TMath::Abs(lep_eta[j])<1.37) || (TMath::Abs(lep_eta[j])>1.52))){
                            if ((TMath::Abs(lep_trackd0pvunbiased[j])/lep_tracksigd0pvunbiased[j]<5) && (TMath::Abs(lep_z0[j]*TMath::Sin(leptemp.Theta()))<0.5)){
                                goodlep_n = goodlep_n + 1;
                                goodlep_index[lep_index] = j;
                                lep_index++;
                            }
                        }
                        // muon selection
                        if ((lep_type[j]==13) && (TMath::Abs(lep_eta[j])<2.5)){
                            if ((TMath::Abs(lep_trackd0pvunbiased[j])/lep_tracksigd0pvunbiased[j]<3) && (TMath::Abs(lep_z0[j]*TMath::Sin(leptemp.Theta()))<0.5)){
                                goodlep_n = goodlep_n + 1;
                                goodlep_index[lep_index] = j;
                                lep_index++;
                            }
                        }
                    }
                }//End_Tight
                
            }
            
            //Exactly two good leptons, leading lepton with pT > 22 GeV and the subleading lepton with pT > 15 GeV
            if (goodlep_n==2){
                       
                int goodlep1_index = goodlep_index[0];
                int goodlep2_index = goodlep_index[1];
                
                if (lep_pt[goodlep1_index]>22000.){
                    
                    //two different-flavour opposite-sign leptons
                    if (lep_type[goodlep1_index] != lep_type[goodlep2_index]){
                        if (lep_charge[goodlep1_index]*lep_charge[goodlep2_index]<0){
                            
                            //TLorentzVector Definitions
                            TLorentzVector Lepton_1  = TLorentzVector();
                            TLorentzVector Lepton_2  = TLorentzVector();
                            TLorentzVector      MeT  = TLorentzVector();
  
                            Lepton_1.SetPtEtaPhiE(lep_pt[goodlep1_index]/1000., lep_eta[goodlep1_index], lep_phi[goodlep1_index],lep_E[goodlep1_index]/1000.);
                            Lepton_2.SetPtEtaPhiE(lep_pt[goodlep2_index]/1000., lep_eta[goodlep2_index], lep_phi[goodlep2_index],lep_E[goodlep2_index]/1000.);
                            MeT.SetPtEtaPhiE(*met_et/1000., 0, *met_phi , *met_et/1000.);
                            
                            TLorentzVector Lepton_12 = TLorentzVector();
                            Lepton_12 = Lepton_1 + Lepton_2;
                            
                            float mLL = Lepton_12.Mag(); //The invariant mass of the two leptons

                            float ptLL = Lepton_12.Pt(); //Transverse momentum of the dilepton system
                            
                            Float_t MET = *met_et/1000.; // Missing transverse momentum
                            
                            //Azimuthal angle between the two leptons
                            float dPhi_LL = TMath::Abs(lep_phi[goodlep1_index]-lep_phi[goodlep2_index]);
                            dPhi_LL = dPhi_LL < TMath::Pi() ? dPhi_LL : 2*TMath::Pi() - dPhi_LL;
                            
                            //Azimuthal angle between E_T^miss and the dilepton system
                            float dPhiLLmet = TMath::Abs(Lepton_12.Phi() - MeT.Phi());
                            dPhiLLmet    = dPhiLLmet < TMath::Pi() ? dPhiLLmet : 2*TMath::Pi() - dPhiLLmet;
                            
                            //dilepton transverse mass
                            float mt = sqrt(2*Lepton_12.Pt()*MeT.Et()*(1-cos(Lepton_12.DeltaPhi(MeT))));
                            
                            // Preselection of good jets
                            int goodjet_n = 0;
                            int goodbjet_n = 0;
                            
                            int goodjet_index[*jet_n];
                            int jet_index = 0;
                            
                            int goodbjet_index[*jet_n];
                            int bjet_index = 0;
                            
                            for (unsigned int j=0; j<*jet_n; j++){
                                
                                if ((jet_pt[j]>20000.) && (TMath::Abs(jet_eta[j])<2.5)){
                                    
                                    //JVT cleaning
                                    bool jvt_pass = true;
                                    
                                    if ((jet_pt[j]<60000.) && (TMath::Abs(jet_eta[j])<2.4) && (jet_jvt[j]<0.59)) {jvt_pass=false;}
                                    
                                    if (jvt_pass){
                                        
                                        // cut on 85% WP
                                        if (jet_MV2c10[j] > 0.1758475){
                                            goodbjet_n++;
                                            goodbjet_index[bjet_index] = j;
                                            bjet_index++;
                                        }
                                        
                                        if (jet_pt[j]>30000.){
                                            goodjet_n++;
                                            goodjet_index[jet_index] = j;
                                            jet_index++;
                                        }
                                    }
                                    
                                }
                            }
                            
                            // fill histogram before any cut
                            double names_of_global_variable_pre[]={mLL, ptLL, dPhi_LL, dPhiLLmet, MET, mt, (double)goodjet_n,
                                                                   (double)goodbjet_n};
                            TString histonames_of_global_variable_pre[]={"histI_mLL", "histI_ptLL", "histI_dPhi_LL", "histI_dPhiLLmet",
                                                                         "histI_etmiss", "histI_mt", "histI_n_jets", "histI_n_bjets"};
                            int length_global_pre = arraysize(names_of_global_variable_pre);
                            for (unsigned int j=0; j<length_global_pre; j++ ){
                                FillHistogramsGlobal(names_of_global_variable_pre[j], weight , histonames_of_global_variable_pre[j]);
                            }
                           
                            
                            //  remove low mass meson resonances and DY events; ggF regions, at least 1 jet
                            if ( (mLL > 10) && (goodjet_n <= 1) && (MET > 20) ){
                                
                                if (mLL < 55){
                                    
                                    if (ptLL > 30){
                                        
                                        if (dPhi_LL < 1.8){
                                            
                                            if (dPhiLLmet > TMath::Pi()/2){
                                                
                                                if (goodbjet_n ==0){
                                                    
                                                    
                                                    //Start to fill histograms : definitions of x-axis variables
                                                    
                                                    double names_of_global_variable[]={mLL, ptLL, dPhi_LL, dPhiLLmet, MET, mt, (double)goodjet_n,
                                                                                       (double)goodbjet_n};
                                                    double names_of_leadlep_variable[]={Lepton_1.Pt(), Lepton_1.Eta(), Lepton_1.E(), Lepton_1.Phi(),
                                                                                        (double)lep_charge[goodlep1_index], (double)lep_type[goodlep1_index]};
                                                    double names_of_subleadlep_variable[]={Lepton_2.Pt(), Lepton_2.Eta(), Lepton_2.E(), Lepton_2.Phi(),
                                                                                           (double)lep_charge[goodlep2_index], (double)lep_type[goodlep2_index]};
                                                    
                                                    //Start to fill histograms : definitions of histogram names
                                                    
                                                    TString histonames_of_global_variable[]={"hist_mLL", "hist_ptLL", "hist_dPhi_LL", "hist_dPhiLLmet",
                                                                                             "hist_etmiss", "hist_mt", "hist_n_jets", "hist_n_bjets"};
                                                    TString histonames_of_leadlep_variable[]={"hist_leadleptpt", "hist_leadlepteta", "hist_leadleptE",
                                                                                              "hist_leadleptphi", "hist_leadleptch", "hist_leadleptID"};
                                                    TString histonames_of_subleadlep_variable[]={"hist_subleadleptpt", "hist_subleadlepteta", "hist_subleadleptE",
                                                                                                 "hist_subleadleptphi", "hist_subleadleptch", "hist_subleadleptID"};
                                                    int length_global = arraysize(names_of_global_variable);
                                                    int length_leadlep = arraysize(names_of_leadlep_variable);
                                                    int length_subleadlep = arraysize(names_of_subleadlep_variable);
                                                    
                                                    // Fill histograms
                                                    
                                                    for (unsigned int j=0; j<length_global; j++){
                                                        FillHistogramsGlobal(names_of_global_variable[j], weight , histonames_of_global_variable[j]);
                                                    }
                                                    
                                                    for (unsigned int j=0; j<length_leadlep; j++){
                                                        FillHistogramsLeadlept(names_of_leadlep_variable[j], weight , histonames_of_leadlep_variable[j]);
                                                    }
                                                    
                                                    for (unsigned int j=0; j<length_subleadlep; j++){
                                                        FillHistogramsSubleadlept(names_of_subleadlep_variable[j], weight, histonames_of_subleadlep_variable[j]);
                                                    }
                                                    
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    
    // Analysis results are saved
    Terminate(sample[i]);
    
    //reset the histograms
    reset_histograms();
    
    // Reset fchain
    fchain->Reset();
}

 +++ Starting PROOF-Lite with 2 workers +++


Opening connections to workers: OK (2 workers)                 


PROOF set to parallel mode (2 workers)


Setting up worker servers: OK (2 workers)                 


Muestra a Analizar: dataA # Events: 668152
Writting with name option: dataA
Muestra a Analizar: dataB # Events: 2459370
Writting with name option: dataB
Muestra a Analizar: dataC # Events: 3587872
Writting with name option: dataC
Muestra a Analizar: dataD # Events: 5490396
Writting with name option: dataD


-------------
Después de analizar dataA.root, dataB.root, dataC.root y dataD.root, se deben agragar las muestras con el comando: hadd data.root dataA.root dataB.root dataC.root dataD.root

In [8]:
if (fileExists("Analysis/Output_HWWAnalysis/data.root") != true){
    if ((fileExists("Analysis/Output_HWWAnalysis/dataA.root") == true) &&
       (fileExists("Analysis/Output_HWWAnalysis/dataB.root") == true)  &&
       (fileExists("Analysis/Output_HWWAnalysis/dataC.root") == true)  &&
       (fileExists("Analysis/Output_HWWAnalysis/dataD.root") == true)) {
           chdir("Analysis/Output_HWWAnalysis/");
           system("hadd data.root dataA.root dataB.root dataC.root dataD.root");
           chdir("../../"); 
       }
}

hadd Target file: data.root
hadd compression setting for all output: 1
hadd Source file 1: dataA.root
hadd Source file 2: dataB.root
hadd Source file 3: dataC.root
hadd Source file 4: dataD.root
hadd Target path: data.root:/


--------------
<h3>Plotting</h3>

Vamos a incluir cabeceras y funciones creadas para obtener los plots finales

In [9]:
#include "Plotting/Plotting.h"
#include "Plotting/Plotting.C"

<p>Verificamos que existan los .root de cada muestra analizada (necesario para los plots finales)</p>

In [10]:
sample.clear();
array_samples(sample,0); //Returns all sample names
bool file_exist = true;
unsigned int n = 0;
unsigned int m = sample.size();
while ((file_exist == true) || (n != m)){
    string name = sample[n];
    string path = "Analysis/Output_HWWAnalysis/"+name+".root";
    if (fileExists(path) != true){
        file_exist = false;
    }
    n++;
}

In [11]:
if (file_exist){
    unsigned int l = 10064;   //luminosity
    string path = "Analysis/Output_HWWAnalysis";
    Plotting *m = new Plotting();
    m->SetLumi(l);
    m->SetOption("HWWAnalysis");
    m->SetInputLocation(path);
    m->run();
}

Analysis option found, proceeding...
Reading list of histograms from: Plotting/list_histos/HistoList_HWWAnalysis.txt 
 
Analysis: HWWAnalysis, reading input from: Plotting/inputfiles/Files_HWW.txt 
 
Reading files from: Analysis/Output_HWWAnalysis 
 
=====processing HWWAnalysis=====


Plotting histogram: histI_dPhiLLmet


Info in <TCanvas::Print>: png file Plotting/histograms/histI_dPhiLLmet.png has been created
Info in <TCanvas::Print>: pdf file Plotting/histograms/histI_dPhiLLmet.pdf has been created


Plotting histogram: histI_dPhi_LL


Info in <TCanvas::SaveAs>: ROOT file Plotting/histograms/histI_dPhiLLmet.root has been created
Info in <TCanvas::Print>: png file Plotting/histograms/histI_dPhi_LL.png has been created
Info in <TCanvas::Print>: pdf file Plotting/histograms/histI_dPhi_LL.pdf has been created


Plotting histogram: histI_etmiss


...

Info in <TCanvas::SaveAs>: ROOT file Plotting/histograms/hist_subleadleptch.root has been created
Info in <TCanvas::Print>: png file Plotting/histograms/hist_subleadlepteta.png has been created
Info in <TCanvas::Print>: pdf file Plotting/histograms/hist_subleadlepteta.pdf has been created


Plotting histogram: hist_subleadleptphi


Info in <TCanvas::SaveAs>: ROOT file Plotting/histograms/hist_subleadlepteta.root has been created
Info in <TCanvas::Print>: png file Plotting/histograms/hist_subleadleptphi.png has been created
Info in <TCanvas::Print>: pdf file Plotting/histograms/hist_subleadleptphi.pdf has been created


Plotting histogram: hist_subleadleptpt


Info in <TCanvas::SaveAs>: ROOT file Plotting/histograms/hist_subleadleptphi.root has been created
Info in <TCanvas::Print>: png file Plotting/histograms/hist_subleadleptpt.png has been created
Info in <TCanvas::Print>: pdf file Plotting/histograms/hist_subleadleptpt.pdf has been created
Info in <TCanvas::SaveAs>: ROOT file Plotting/histograms/hist_subleadleptpt.root has been created
